![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### ODPi Egeria Hands-On Lab
# Welcome to the Configuring Metadata Repositories Lab

## Introduction

ODPi Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogues and platforms together so they can share information about data and technology.  This information is called metadata.

In this hands-on lab you will learn how to configure the metadata servers used by [Coco Pharmaceuticals](https://opengovernance.odpi.org/coco-pharmaceuticals/).

## The scenario

Coco Pharmaceuticals is going through a major business transformation that requires them to drastically reduce their cycle times, collaborate laterally across the different parts of the business and react quickly to the changing needs of their customers. (See [this link](https://opengovernance.odpi.org/coco-pharmaceuticals/) for the background to this transformation).

Part of the changes needed to the IT systems that support the business is the roll out of a distributed metadata and governance capability that is provided by ODPi Egeria.

[Gary Geeke](https://opengovernance.odpi.org/coco-pharmaceuticals/personas/gary-geeke.html) is the IT Infrastructure leader at Coco Pharmaceuticals.

![Gary Geeke](https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/gary-geeke.png)

In this hands-on lab Gary is configuring the metadata servers ready for deployment.  Gary's userId is `garygeeke`.

In [ ]:
import requests

adminUserId     = "garygeeke"

He needs to define the metadata servers for Coco Pharmaceuticals.

In [ ]:
organizationName = "Coco Pharmaceuticals"
serverType       = "Open Metadata Server"

## Metadata management landscape

Gary has decided to deploy a separate metadata server for each part of the organization that owns assets.  They are as follows:

* cocoMDS1 - Data Lake Operations - used to manage the data in the data lake.
* cocoMDS2 - Governance - used by all of the governance teams to operate the governance programs.
* cocoMDS3 - Research - used by the research teams who are developing new treatments.
* cocoMDS4 - Data Lake Users - used by general business users and the executive team to access data from the data lake.
* cocoMDS5 - Business Systems - used to record information about the operational business systems such as procurements, sales, human resources and finance.
* cocoMDS6 - Manufacturing - used by the warehouse, manufacturing and distribution teams.
* cocoMDSx - Development - used by the development teams building new IT capablity.
* cocoEDGE*i* - Manufacturing sensors edge node servers (many of them).

These metadata servers will each be configured in later parts of the hands-on lab, but first there are decisons to be made about the platform that the servers will run on and how they will be connected together.

### OMAG Server Platforms

The Coco Pharmaceuticals' metadata servers along with the governance servers that interact with them
must be hosted on at least one OMAG Server Platform.

Most of the metadata servers are pretty stable and can share an OMAG Server Platform.
The data lake however requires a lot of active governance so Gary chooses to put all of the metadata and governance servers for the data lake onto their own plaform.  The development team requested that their infrastructure is completely separate from the operational systems,
so they are given their own server platform.  Finally each of the edge servers will run an OMAG Server Platform to support
their metadata server.

Figure 1 shows which metadata servers will sit in each platform.

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

Below are the host name and port number where the core, data lake and development platforms will run. 

In [ ]:
import os

corePlatformURL     = os.environ.get('corePlatformURL','http://localhost:8080') 
dataLakePlatformURL = os.environ.get('dataLakePlatformURL','http://localhost:8081') 
devPlatformURL      = os.environ.get('devPlatformURL','http://localhost:8082')

The sensor edge node servers used to monitor the warehouse operation and manufacturing process each have their own platform and are not yet included in this notebook.

### Open Metadata Repository Cohorts

Metadata servers communicate via open metadata repository cohorts.  A server can become a member of none, one or many cohorts.  Once it has joined a cohort it can exchange metadata with the other members of that cohort.  So the cohorts define scopes of sharing.

Gary decides to begin with three open metadata repository cohorts:

* **cocoCohort** - The production cohort contains all of the metadata servers that are used to run, coordinate and govern the business.
* **devCohort** - The development cohort where the development teams are building and testing new capablity.  Much of their metadata describes the software components under construction and the governance of the software development lifecycle.
* **iotCohort** - The IoT cohort used to manage the sensors and robots in the manufacturing systems.  The metadata produced by the sensors and robots is only of interest to the manufactuing and governance team.

Figure 2 shows which metadata servers belong to each cohort.

![Figure 2](../images/coco-pharmaceuticals-systems-metadata-servers.png)
> **Figure 2:** Membership of Coco Pharmaceuticals' cohorts

Below are the names of the three cohorts.

In [ ]:
cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"

At the heart of each cohort is an event topic.  By default, ODPi Egeria uses [Apache Kafka](https://kafka.apache.org/) topics.
The metadata servers will need to be configured with the host name and port where Kafka is running.
The command below pulls the value from an environment variable called `eventBusURLroot` with a default value of
`localhost:59092`.

In [ ]:
eventBusURLroot = os.environ.get('eventBusURLroot', 'localhost:59092')

## Access services

The table below shows which access services are needed by each server.


| Access Service       | cocoMDS1 | cocoMDS2 | cocoMDS3 | cocoMDS4 | cocoMDS5 | cocoMDS6 | cocoMDSx | cocoEDGE*i* |
| :------------------- | :------: | :------: | :------: | :------: | :------: | :------: | :------: | :---------: |
| asset-catalog        |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| asset-consumer       |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| asset-owner          |   Yes    |   No     |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| community-profile    |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| glossary-view        |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| -------------------  |  ------  |  ------  | ------   |  ------  |  ------  |  ------  |  ------  | ----------  |
| data-science         |   No     |   No     |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| -------------------  |  ------  |  ------  | ------   |  ------  |  ------  |  ------  |  ------  | ----------  |
| subject-area         |   No     |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| -------------------  |  ------  |  ------  | ------   |  ------  |  ------  |  ------  |  ------  | ----------  |
| governance-program   |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| data-privacy         |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| security-officer     |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| asset-lineage        |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| discovery-engine     |   Yes    |   No     |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| stewardship-action   |   Yes    |   No     |   Yes    |   No     |   No     |   No     |   Yes    |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| data-engine          |   Yes    |   No     |   No     |   No     |   No     |   Yes    |   No     |     Yes     |
| data-platform        |   Yes    |   No     |   No     |   No     |   No     |   Yes    |   No     |     Yes     |
| governance-engine    |   Yes    |   No     |   No     |   No     |   No     |   Yes    |   No     |     No      |
| information-view     |   Yes    |   No     |   No     |   No     |   No     |   Yes    |   No     |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| it-infrastructure    |   No     |   Yes    |   No     |   No     |   No     |   Yes    |   Yes    |     No      |
| project-management   |   No     |   Yes    |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| software-development |   No     |   No     |   No     |   No     |   No     |   No     |   Yes    |     No      |
| devops               |   No     |   No     |   No     |   No     |   No     |   No     |   Yes    |     No      |
| digital-architecture |   No     |   No     |   No     |   No     |   No     |   No     |   Yes    |     No      |
| data-process         |   No     |   No     |   No     |   No     |   No     |   No     |   Yes    |     No      |
| :------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |



## ODPi Egeria Server Configuration

Open metadata servers are configured using REST API calls to an OMAG Server Platform.  Each call either defines a default value or configures a service that must run within the server when it is started.

As each configuration call is made, the server platform builds up a [configuration document](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/configuration-document.html) with the values passed.  When the configuration is finished, the configuration document will have all of the information needed to start the server.

The configuration document will be deployed with the server platform that is hosting the server.  When a request is made to this server platform to start the server, it reads the configuration document and initializes the server with the apporpiate services.

A server can be configured by any OMAG Server Platform - it does not have to be the same platform where the server will run.  For this hands on lab we will use a server platform that is running on the local machine (`localhost`) and with the default port of `8080` to configure all of Coco Pharmaceutical's metadata servers.

In [ ]:
adminPlatformURL = corePlatformURL

The URLs for the configuration REST APIs have a common structure and begin with the following root:

In [ ]:
adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'

Finally, to ensure that a caller can not request too much metadata in a single request, it is possible to set a maximum page size for requests that return a list of items.  The maximum page size puts a limit on the number of items that can be requested.

In [ ]:
maxPageSize = '100'

## Starting the OMAG Server Platform

To start the server platform, open a console window and go to the ODPi Egeria server folder.
Then issue the `java -jar *jar command` to start the server platform.  Figure 3 shows the output of the server platform as it starts.

----
![Figure 3](../images/omag-server-platform-ready.png)

----
> **Figure 3:** OMAG Server Platform start up messages


Once you see the message `<date time> OMAG server platform ready for configuration` it is ready to begin.

What follows are descriptions and coded requests to configure each server.  There are a lot of common steps involved in configuring a metadata server.  These steps are broken down and explained in detail for the first server they appear in.  For subsequent servers, the common steps are batched together and there is little explanation.  Therefore it is suggested that you follow the workbook sequentially the first time through to get familiar with the commands.

## Configuring cocoMDS1 - Data Lake Operations metadata server

This section configures the `cocoMDS1` server.

In [ ]:
 mdrServerName     = "cocoMDS1"

This server name is passed on every configuration call to identify which configuration document to update with the new configuration.  The configuration document is created automatically on first use.

----
### Setting up names and identifiers

Below are the commands to configure the URL of the platform where the server will run, the maximum page size, the owning organization name and server type.

In [ ]:
import pprint
import json

mdrServerPlatform      = dataLakePlatformURL

print (" ")
print ("Configuring the platform that the server will run on ...")
url = adminCommandURLRoot + mdrServerName + '/server-url-root?url=' + mdrServerPlatform
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the maximum page size ...")
url = adminCommandURLRoot + mdrServerName + '/max-page-size?limit=' + maxPageSize
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's type ...")
url = adminCommandURLRoot + mdrServerName + '/server-type?typeName=' + serverType
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's owning organization ...")
url = adminCommandURLRoot + mdrServerName + '/organization-name?name=' + organizationName
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


----

### Configuring security

There are two levels of security to set up for an ODPi Egeria server: authentication and authorization.


#### Authentication of servers and people

ODPi Egeria recommends that each server has its own identity and that is embedded with each request as part of the transport level security (TLS).  The members of the cohort (and the event topic) then grant access to each other and no-one else.

The identity of the calling user also flows with each request, but this time as a unique string value (typically userId) in the URL of the request.  You can see examples of this in the configuration requests being issued during this hands-on lab as Gary's userId `garygeeke` appears on each request.

The server configuration supports a userId and password for TLS.  The userId is also used when the server is processing requests that originate from an event and so there is no calling user.


In [ ]:
mdrServerUserId   = "cocoMDS1npa"
mdrServerPassword = "cocoMDS1passw0rd"

print (" ")
print ("Configuring the server's userId ...")
url = adminCommandURLRoot + mdrServerName + '/server-user-id?id=' + mdrServerUserId
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's password (optional) ...")
url = adminCommandURLRoot + mdrServerName + '/server-user-password?password=' + mdrServerPassword
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


----
#### Authorization of metadata requests

ODPi Egeria servers also support a metadata security connector that plugs into the server and is called to provide authorization decisions as part of every request.

This connector is configured in the configuration document by passing the **Connection** object that provides the properties needed to create the connecto on the following call ...


In [ ]:
print (" ")
print ("Configuring the server's security connector ...")

url = adminCommandURLRoot + mdrServerName + '/security/connection'
print ("POST " + url)

header={'content-type':'application/json'}
body = {
    "class": "Connection",
    "type": {
        "class": "ElementType",
        "elementTypeId": "114e9f8f-5ff3-4c32-bd37-a7eb42712253",
        "elementTypeName": "Connection",
        "elementTypeVersion": 1,
        "elementTypeDescription": "A set of properties to identify and configure a connector instance.",
        "elementOrigin": "CONFIGURATION"
    },
    "guid": "1213abc7-2b13-4c4e-b840-97c4282f7416",
    "qualifiedName": "CocoPharmaceuticalsMetadataServerSecurityConnector",
    "displayName": "Metadata Server Security Connector",
    "description": "Connector to enforce authorization rules for accessing and updating metadata.",
    "connectorType": {
        "class": "ConnectorType",
        "type": {
            "class": "ElementType",
            "elementTypeId": "954421eb-33a6-462d-a8ca-b5709a1bd0d4",
            "elementTypeName": "ConnectorType",
            "elementTypeVersion": 1,
            "elementTypeDescription": "A set of properties describing a type of connector.",
            "elementOrigin": "LOCAL_COHORT"
        },
        "guid": "1851f73d-e343-abcd-82cb-3918fed81da6",
        "qualifiedName": "CocoPharmaServerSecurityConnectorType",
        "displayName": "Coco Pharmaceuticals Server Security Connector Implementation",
        "description": "This connector ensures only valid and authorized people can access the metadata.",
        "connectorProviderClassName": "org.odpi.openmetadata.metadatasecurity.samples.CocoPharmaServerSecurityProvider"
    }
}

response=requests.post(url, json=body, headers=header)

print ("Response: ")
print (response.json())


----

### Configuring the local repository

The cocoMDS1 metadata server needs a local repository to store metadata about the data and processing occuring in the data lake.

ODPi Egeria supports 2 types of repositories.  One is an in-memory repository that stores metadata in hash maps.  It is useful for demos and testing because a restart of the server results in an empty metadata repository.  However, if you need metadata to persist from one run of the server to the next, you should use the graph repository.

The choice of local repository is made by specifying the local repository mode.  The variables below show the two options.  The `metadataRepositoryType` identfies which one is going to be used in the configuration.


In [ ]:
inMemoryRepositoryOption = "in-memory-repository"
graphRepositoryOption    = "local-graph-repository"

metadataRepositoryType = inMemoryRepositoryOption

This is the call that sets up the local repository.

In [ ]:
print (" ")
print ("Configuring the metadata repository ...")

url = adminCommandURLRoot + mdrServerName + '/local-repository/mode/' + metadataRepositoryType
print ("POST " + url)

response=requests.post(url)

print ("Response: ")
print (response.json())

The metadata inside tha repository is called the **metadata collection**.  It is helpful to assign a meaningful name to the metadata collection as it appears in error handling and displays.

In [ ]:
metadataCollectionName = "Data Lake Catalog"

The code below assigns the metadata collection name to the repository.

In [ ]:

print (" ")
print ("Configuring the short descriptive name of the metadata stored in this server ...")

url = adminCommandURLRoot + mdrServerName + '/local-repository/metadata-collection-name/' + metadataCollectionName
print ("POST " + url)

response=requests.post(url)

print ("Response: ")
print (response.json())


----
One of the important values added to the configuration is the **metadata_collection_id**.  This is the unique identfier for the metadata collection that originates from this server.  It is encoded into ever instance that is created in this repository.

In [ ]:

print (" ")
print ("Retrieving the local metadata collection id ...")

url = adminCommandURLRoot + mdrServerName + '/local-repository/metadata-collection-id'
print ("GET " + url)

response=requests.get(url)

print ("Response: ")
print (response.json())


----
### Setting up the event bus

The server needs to define the event bus it will use to exchange metadata.  This event bus is used to connect to the cohorts and to provide the in / out topics for each of the Open Metadata Access Services (OMASs) - more later.

In [ ]:

print (" ")
print ("Configuring the event bus for this server ...")
url = adminCommandURLRoot + mdrServerName + '/event-bus'
print ("POST " + url)

header={'content-type':'application/json'}
body = {
    "producer": {
        "bootstrap.servers": eventBusURLroot
     },
    "consumer":{
        "bootstrap.servers": eventBusURLroot
    }
}

response=requests.post(url, json=body, headers=header)

print ("Response: ")
print (response.json())


----

### Specifying the cohort

Next we add cocoMDS1 to the main cocoCohort ...


In [ ]:
print (" ")
print ("Configuring the membership of the cohort ...")

url = adminCommandURLRoot + mdrServerName + '/cohorts/' + cocoCohort
print ("POST " + url)

response=requests.post(url)

print ("Response: ")
print (response.json())

----
### Configuring the access services

The final step is to configure the access services.  Access services provide the specialized APIs and events for different tools and personas.  ODPi Egeria provides an initial set of access services, and additional services can be pluggable into the server platform.

To query the choice of access services, use the follow command:

In [ ]:

print (" ")
print ("Retrieving the registered access services ...")
url = adminPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/registered-services/access-services"
print ("GET " + url)

response = requests.get(url)

prettyResponse = json.dumps(response.json(), indent=4)
print ("Response: ")
print (prettyResponse)
print (" ")


----
The `cocoMDS1` server is for the data lake operations.  It needs the access services to support the onboarding and decommissioning of assets along with the access services that supports the different engines that maintain the data lake.

In [ ]:

print (" ")
print ("Configuring the access services for this server ...")
accessServiceURL = adminCommandURLRoot + mdrServerName + '/access-services'
header={'content-type':'application/json'}
nullAccessServiceOptions = {}

serviceURLMarker = 'asset-catalog'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "SupportedZones": ["quarantine", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'asset-consumer'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "SupportedZones": ["quarantine", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'asset-owner'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "DefaultZones": [ "quarantine" ],
  "SupportedZones": ["quarantine", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'community-profile'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "KarmaPointPlateau": "500"
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'glossary-view'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'discovery-engine'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "DefaultZones": [ "quarantine" ],
  "SupportedZones": ["quarantine", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'stewardship-action'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "DefaultZones": [ "quarantine" ],
  "SupportedZones": ["quarantine", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'data-engine'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "DefaultZones": [ "quarantine" ],
  "SupportedZones": ["quarantine", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'data-platform'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "DefaultZones": [ "quarantine" ],
  "SupportedZones": ["quarantine", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'governance-engine'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "DefaultZones": [ "quarantine" ],
  "SupportedZones": ["quarantine", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'information-view'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "DefaultZones": [ "quarantine" ],
  "SupportedZones": ["quarantine", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())


----

## Configuring cocoMDS2 - Governance metadata server

This section configures the `cocoMDS2` server.  This server is configured in a similar way to cocoMDS1 except that is has different Open Metadata Access Services (OMASs) enabled and it joins all of the cohorts.

The code below covers the basic set up of the server properties, security, event bus and local repository.

In [ ]:
mdrServerName          = "cocoMDS2"
mdrServerUserId        = "cocoMDS2npa"
mdrServerPassword      = "cocoMDS2passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Governance Catalog"


print (" ")
print ("Configuring the platform that the server will run on ...")
url = adminCommandURLRoot + mdrServerName + '/server-url-root?url=' + mdrServerPlatform
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the maximum page size ...")
url = adminCommandURLRoot + mdrServerName + '/max-page-size?limit=' + maxPageSize
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the server's type ...")
url = adminCommandURLRoot + mdrServerName + '/server-type?typeName=' + serverType
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the server's owning organization ...")
url = adminCommandURLRoot + mdrServerName + '/organization-name?name=' + organizationName
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the server's userId ...")
url = adminCommandURLRoot + mdrServerName + '/server-user-id?id=' + mdrServerUserId
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the server's password ...")
url = adminCommandURLRoot + mdrServerName + '/server-user-password?password=' + mdrServerPassword
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the server's security connector ...")
url = adminCommandURLRoot + mdrServerName + '/security/connection'
print ("POST " + url)
header={'content-type':'application/json'}
body = {
    "class": "Connection",
    "type": {
        "class": "ElementType",
        "elementTypeId": "114e9f8f-5ff3-4c32-bd37-a7eb42712253",
        "elementTypeName": "Connection",
        "elementTypeVersion": 1,
        "elementTypeDescription": "A set of properties to identify and configure a connector instance.",
        "elementOrigin": "CONFIGURATION"
    },
    "guid": "1213abc7-2b13-4c4e-b840-97c4282f7416",
    "qualifiedName": "CocoPharmaceuticalsMetadataServerSecurityConnector",
    "displayName": "Metadata Server Security Connector",
    "description": "Connector to enforce authorization rules for accessing and updating metadata.",
    "connectorType": {
        "class": "ConnectorType",
        "type": {
            "class": "ElementType",
            "elementTypeId": "954421eb-33a6-462d-a8ca-b5709a1bd0d4",
            "elementTypeName": "ConnectorType",
            "elementTypeVersion": 1,
            "elementTypeDescription": "A set of properties describing a type of connector.",
            "elementOrigin": "LOCAL_COHORT"
        },
        "guid": "1851f73d-e343-abcd-82cb-3918fed81da6",
        "qualifiedName": "CocoPharmaServerSecurityConnectorType",
        "displayName": "Coco Pharmaceuticals Server Security Connector Implementation",
        "description": "This connector ensures only valid and authorized people can access the metadata.",
        "connectorProviderClassName": "org.odpi.openmetadata.metadatasecurity.samples.CocoPharmaServerSecurityProvider"
    }
}
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the metadata repository ...")
url = adminCommandURLRoot + mdrServerName + '/local-repository/mode/' + metadataRepositoryType
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the short descriptive name of the metadata stored in this server ...")
url = adminCommandURLRoot + mdrServerName + '/local-repository/metadata-collection-name/' + metadataCollectionName
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the event bus for this server ...")
url = adminCommandURLRoot + mdrServerName + '/event-bus'
print ("POST " + url)
header={'content-type':'application/json'}
body = {
    "producer": {
        "bootstrap.servers": eventBusURLroot
     },
    "consumer":{
        "bootstrap.servers": eventBusURLroot
    }
}
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the membership of all cohorts ...")
url = adminCommandURLRoot + mdrServerName + '/cohorts/' + cocoCohort
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())
url = adminCommandURLRoot + mdrServerName + '/cohorts/' + devCohort
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())
url = adminCommandURLRoot + mdrServerName + '/cohorts/' + iotCohort
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the access services for this server ...")
accessServiceURL = adminCommandURLRoot + mdrServerName + '/access-services'

serviceURLMarker = 'asset-catalog'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'asset-consumer'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'community-profile'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "KarmaPointPlateau": "500"
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'glossary-view'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'subject-area'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'governance-program'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'data-privacy'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'security-officer'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'asset-lineage'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'it-infrastructure'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'project-management'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())


----

## Configuring cocoMDS3 - Research

Server cocoMDS3 is used by the research teams who are developing new treatments.
These teams are working with their own assets as well as assets coming from the data lake.
They have their own discovery server to automate analysis of their data.
They are also creating new data science models that they run by hand, or deploy to the discovery server.

In [ ]:
mdrServerName          = "cocoMDS3"
mdrServerUserId        = "cocoMDS3npa"
mdrServerPassword      = "cocoMDS3passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Research Catalog"

print (" ")
print ("Configuring the platform that the server will run on ...")
url = adminCommandURLRoot + mdrServerName + '/server-url-root?url=' + mdrServerPlatform
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the maximum page size ...")
url = adminCommandURLRoot + mdrServerName + '/max-page-size?limit=' + maxPageSize
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's type ...")
url = adminCommandURLRoot + mdrServerName + '/server-type?typeName=' + serverType
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's owning organization ...")
url = adminCommandURLRoot + mdrServerName + '/organization-name?name=' + organizationName
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's userId ...")
url = adminCommandURLRoot + mdrServerName + '/server-user-id?id=' + mdrServerUserId
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's password ...")
url = adminCommandURLRoot + mdrServerName + '/server-user-password?password=' + mdrServerPassword
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's security connector ...")
url = adminCommandURLRoot + mdrServerName + '/security/connection'
print ("POST " + url)
header={'content-type':'application/json'}
body = {
    "class": "Connection",
    "type": {
        "class": "ElementType",
        "elementTypeId": "114e9f8f-5ff3-4c32-bd37-a7eb42712253",
        "elementTypeName": "Connection",
        "elementTypeVersion": 1,
        "elementTypeDescription": "A set of properties to identify and configure a connector instance.",
        "elementOrigin": "CONFIGURATION"
    },
    "guid": "1213abc7-2b13-4c4e-b840-97c4282f7416",
    "qualifiedName": "CocoPharmaceuticalsMetadataServerSecurityConnector",
    "displayName": "Metadata Server Security Connector",
    "description": "Connector to enforce authorization rules for accessing and updating metadata.",
    "connectorType": {
        "class": "ConnectorType",
        "type": {
            "class": "ElementType",
            "elementTypeId": "954421eb-33a6-462d-a8ca-b5709a1bd0d4",
            "elementTypeName": "ConnectorType",
            "elementTypeVersion": 1,
            "elementTypeDescription": "A set of properties describing a type of connector.",
            "elementOrigin": "LOCAL_COHORT"
        },
        "guid": "1851f73d-e343-abcd-82cb-3918fed81da6",
        "qualifiedName": "CocoPharmaServerSecurityConnectorType",
        "displayName": "Coco Pharmaceuticals Server Security Connector Implementation",
        "description": "This connector ensures only valid and authorized people can access the metadata.",
        "connectorProviderClassName": "org.odpi.openmetadata.metadatasecurity.samples.CocoPharmaServerSecurityProvider"
    }
}
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the metadata repository ...")
url = adminCommandURLRoot + mdrServerName + '/local-repository/mode/' + metadataRepositoryType
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the short descriptive name of the metadata stored in this server ...")
url = adminCommandURLRoot + mdrServerName + '/local-repository/metadata-collection-name/' + metadataCollectionName
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the event bus for this server ...")
url = adminCommandURLRoot + mdrServerName + '/event-bus'
print ("POST " + url)
header={'content-type':'application/json'}
body = {
    "producer": {
        "bootstrap.servers": eventBusURLroot
     },
    "consumer":{
        "bootstrap.servers": eventBusURLroot
    }
}
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the membership of the cocoCohort ...")
url = adminCommandURLRoot + mdrServerName + '/cohorts/' + cocoCohort
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the access services for this server ...")
accessServiceURL = adminCommandURLRoot + mdrServerName + '/access-services'
header={'content-type':'application/json'}

serviceURLMarker = 'asset-catalog'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
body = {
  "SupportedZones": [ "personal-files", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())

serviceURLMarker = 'asset-consumer'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())

serviceURLMarker = 'asset-owner'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
body = {
  "DefaultZones": [ "personal-files" ],
  "SupportedZones": [ "personal-files", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())

serviceURLMarker = 'community-profile'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "KarmaPointPlateau": "500"
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'glossary-view'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'data-science'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'subject-area'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'discovery-engine'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
body = {
  "DefaultZones": [ "personal-files" ],
  "SupportedZones": [ "personal-files", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())

serviceURLMarker = 'stewardship-action'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
body = {
  "DefaultZones": [ "personal-files" ],
  "SupportedZones": [ "personal-files", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())

serviceURLMarker = 'project-management'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
body = {
  "DefaultZones": [ "personal-files" ],
  "SupportedZones": [ "personal-files", "clinical-trials", "research", "data-lake", "trash-can" ]
}
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())


## Configuring cocoMDS4 - Data Lake Users

Server cocoMDS4 used by general business users and the executive team to access data from the data lake.
It does not have a repository of its own.  Instead it issues federated queries to the other repositories in the `cocoCohort`.

In [ ]:
mdrServerName          = "cocoMDS4"
mdrServerUserId        = "cocoMDS4npa"
mdrServerPassword      = "cocoMDS4passw0rd"
mdrServerPlatform      = dataLakePlatformURL
metadataCollectionName = "Data Lake Catalog"

print (" ")
print ("Configuring the platform that the server will run on ...")
url = adminCommandURLRoot + mdrServerName + '/server-url-root?url=' + mdrServerPlatform
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the maximum page size ...")
url = adminCommandURLRoot + mdrServerName + '/max-page-size?limit=' + maxPageSize
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's type ...")
url = adminCommandURLRoot + mdrServerName + '/server-type?typeName=' + serverType
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's owning organization ...")
url = adminCommandURLRoot + mdrServerName + '/organization-name?name=' + organizationName
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's userId ...")
url = adminCommandURLRoot + mdrServerName + '/server-user-id?id=' + mdrServerUserId
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's password ...")
url = adminCommandURLRoot + mdrServerName + '/server-user-password?password=' + mdrServerPassword
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's security connector ...")
url = adminCommandURLRoot + mdrServerName + '/security/connection'
print ("POST " + url)
header={'content-type':'application/json'}
body = {
    "class": "Connection",
    "type": {
        "class": "ElementType",
        "elementTypeId": "114e9f8f-5ff3-4c32-bd37-a7eb42712253",
        "elementTypeName": "Connection",
        "elementTypeVersion": 1,
        "elementTypeDescription": "A set of properties to identify and configure a connector instance.",
        "elementOrigin": "CONFIGURATION"
    },
    "guid": "1213abc7-2b13-4c4e-b840-97c4282f7416",
    "qualifiedName": "CocoPharmaceuticalsMetadataServerSecurityConnector",
    "displayName": "Metadata Server Security Connector",
    "description": "Connector to enforce authorization rules for accessing and updating metadata.",
    "connectorType": {
        "class": "ConnectorType",
        "type": {
            "class": "ElementType",
            "elementTypeId": "954421eb-33a6-462d-a8ca-b5709a1bd0d4",
            "elementTypeName": "ConnectorType",
            "elementTypeVersion": 1,
            "elementTypeDescription": "A set of properties describing a type of connector.",
            "elementOrigin": "LOCAL_COHORT"
        },
        "guid": "1851f73d-e343-abcd-82cb-3918fed81da6",
        "qualifiedName": "CocoPharmaServerSecurityConnectorType",
        "displayName": "Coco Pharmaceuticals Server Security Connector Implementation",
        "description": "This connector ensures only valid and authorized people can access the metadata.",
        "connectorProviderClassName": "org.odpi.openmetadata.metadatasecurity.samples.CocoPharmaServerSecurityProvider"
    }
}
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the event bus for this server ...")
url = adminCommandURLRoot + mdrServerName + '/event-bus'
print ("POST " + url)
header={'content-type':'application/json'}
body = {
    "producer": {
        "bootstrap.servers": eventBusURLroot
     },
    "consumer":{
        "bootstrap.servers": eventBusURLroot
    }
}
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the membership of the cocoCohort ...")
url = adminCommandURLRoot + mdrServerName + '/cohorts/' + cocoCohort
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the access services for this server ...")
accessServiceURL = adminCommandURLRoot + mdrServerName + '/access-services'
header={'content-type':'application/json'}
dataLakeAccessServiceOptions = {
  "SupportedZones": [ "data-lake" ]
}

serviceURLMarker = 'asset-catalog'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=body, headers=header)
print ("Response: ")
print (response.json())

serviceURLMarker = 'asset-consumer'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=dataLakeAccessServiceOptions, headers=header)
print ("Response: ")
print (response.json())

serviceURLMarker = 'asset-owner'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=dataLakeAccessServiceOptions, headers=header)
print ("Response: ")
print (response.json())

serviceURLMarker = 'community-profile'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
accessServiceOptions = {
  "KarmaPointPlateau": "500"
}
response=requests.post(url, json=accessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'glossary-view'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())

serviceURLMarker = 'data-science'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=dataLakeAccessServiceOptions, headers=header)
print ("Response: ")
print (response.json())

serviceURLMarker = 'subject-area'
url = accessServiceURL + '/' + serviceURLMarker
print ("POST " + url)
response=requests.post(url, json=nullAccessServiceOptions, headers=header )
print ("Response: ")
print (response.json())



----

## Configuring cocoMDS5 - Business Systems

Server cocoMDS5 is a repository proxy to an ETL tool called `iisCore01`.  This ETL tool is well established in Coco Pharmaceuticals and has a built in metadata repository that contains information about their operational business systems such as procurements, sales, human resources and finance.

This ETL tool has its own user interface and services so the OMASs are not enabled.

In [ ]:
mdrServerName          = "cocoMDS5"
mdrServerUserId        = "cocoMDS5npa"
mdrServerPassword      = "cocoMDS5passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Business Systems Catalog"

----

## Configuring cocoMDS6 - Manufacturing

Server cocoMDS6 is the repository server used by the warehouse, manufacturing and distribution teams.  It supports the systems for this part of the organization and acts as a hub for monitoring the IoT environment.

In [ ]:
mdrServerName          = "cocoMDS6"
mdrServerUserId        = "cocoMDS6npa"
mdrServerPassword      = "cocoMDS6passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Manufacturing Catalog"

----

## Configuring cocoMDSx - Development

Server cocoMDSx is used by the development teams building new IT capablity.  It will hold all of the software component assets and servers used for development and devOps.  The development teams have their own OMAG Server Platform and cohort called 'devCohort'.

In [ ]:
mdrServerName          = "cocoMDSx"
mdrServerUserId        = "cocoMDSxnpa"
mdrServerPassword      = "cocoMDSxpassw0rd"
mdrServerPlatform      = devPlatformURL
metadataCollectionName = "Development Catalog"

----